## Prompt ChatGLM3
#### [·] to classify whether a question belongs to medical topic.

In [4]:
import random
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel
import pandas as pd

tokenizer = AutoTokenizer.from_pretrained("ChatGLM3", trust_remote_code=True)
model = AutoModel.from_pretrained("ChatGLM3", trust_remote_code=True).half().cuda()
model = model.eval()

Setting eos_token is not supported, use the default one.
Setting pad_token is not supported, use the default one.
Setting unk_token is not supported, use the default one.


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [16]:
questions = ["今天星期几？",
             "夏天适合去哪里旅游？",
             "肚子好痛啊！该吃什么药？",
             "湿气重怎么办？",
             "上火有什么办法吗？",
             "一觉醒来着凉了！",
             "感冒怎么治？",
             "感冒药还有库存吗？",
             "你们店里有什么药？",
             "高血压会遗传吗？",
             "脾胃虚是因为肝火旺吗？"]

In [53]:
def prompt(question):
    return f"请判断下列提问是否有关病症、医药、药物、治疗中的任何一类，如果是请回答1，否则请回答0，问题如下：{question}" 

In [33]:
for q in questions:
    response, history = model.chat(tokenizer, prompt(q), history=[])
    print("Response:",response,"Ask:",q)

Response: False Ask: 今天星期几？
Response: 这个问题与病症、医药、药物、治疗中的任何一类无关，所以我的回答是False。 Ask: 夏天适合去哪里旅游？
Response: True。 Ask: 肚子好痛啊！该吃什么药？
Response: True Ask: 湿气重怎么办？
Response: True Ask: 上火有什么办法吗？
Response: False Ask: 一觉醒来着凉了！
Response: True Ask: 感冒怎么治？
Response: True Ask: 感冒药还有库存吗？
Response: True Ask: 你们店里有什么药？
Response: True Ask: 高血压会遗传吗？
Response: False Ask: 脾胃虚是因为肝火旺吗？


In [34]:
data = pd.read_csv("data_0_1_final.csv")
data

,problem,label
0,"老师，为什么用scanf函数时，如果写成scanf(""%d\n"",a),按回车后就没有结果出来呢？",0
1,老师，vc6++下载发现有病毒。能解决一下吗？,0
2,1）既然无参数宏定义是用标识符来代表字符串的，符号常量也是用标识符命名常量，这两个有什么区别...,0
3,胡老师，你好，我在学习过程中遇到几个问题，想要清教你一下。\n第一，关于void。课本上说v...,0
4,该程序打印学生课程成绩并且计算学生平均成绩和课程平均成绩\n但成绩显示，但为什么不显示平均成...,0
...,...,...
39995,怀孕试纸可以测出来吗？如果怀孕在月经推迟一个星期之内也可以测出的吧,1
39996,发热在每天下午开始高热后来低热检查过一些常规检查都检查了没有查出问题有什么好的地方看,1
39997,哦，这样子的呀，那先谢谢了，到时我就担心服完药一周后还没来月经，然后我再去检查还说没有怀孕那...,1
39998,前一段时间胃病，好像是胃炎，胃胀痛，恶心，呕吐，现在症状好了，出现了嘴苦，是不是胃热引起的，怎么办,1


In [62]:
sample = data.sample(frac=0.0025)
correct = 0

for idx,row in tqdm(sample.iterrows()):
    q = row["problem"]
    response, history = model.chat(tokenizer, prompt(row["problem"]), history=[])
    try:
        if int(response[0]) == row["label"]:
            correct += 1
        else:
            pass
    except:
        print(response)
        print(type(response))
        print(row["label"])
        print(type(row["label"]))
print("Accuracy:",correct/len(sample))

100it [00:12,  8.30it/s]

Accuracy: 0.84
